In [ ]:
import os
import time
import urllib.parse
import requests
from bs4 import BeautifulSoup

In [ ]:
session = requests.Session()
url = 'https://pip.moi.gov.tw/Publicize/Info/E4020'

# （1）先 GET 頁面取得 token
resp = session.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'html.parser')
token = soup.find('input', {'name': '__RequestVerificationToken'})['value']

In [ ]:
# 4. Base payload
payload = {
    # 下列跟 DevTools 裡一模一樣的 key
    "CitySel":   "A",
    "F01Sel":    urllib.parse.quote("住宅供給", encoding="utf-8"),
    "F02Sel":    "TAX02",
    "DDATESel":  "114Q1",
    "City":      "A",
    "F01":       urllib.parse.quote("住宅供給", encoding="utf-8"),
    "F02":       "TAX02",
    "DDATE":     "114Q1",
    "ResponseCommand": urllib.parse.quote("檢視查詢結果", encoding="utf-8"),
    "Command":         "exportCSV",
    # 如果 CommandArgument 不需要就留空字串
    "CommandArgument": "",
    "__RequestVerificationToken": token
}

headers = {
    'Referer': url,
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
}

In [ ]:
download = session.post(url, data=payload)
download.encoding = 'big5'   # 如果網站回傳的是 big5 編碼
filename = f"E1040_{payload['City']}.csv"

with open(filename, 'wb') as f:
    f.write(download.content)

print(f"已下載並儲存為：{filename}")